In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# read metis output file
with open(r'ig_communities_output.metis', 'r') as fp:
    file = fp.readlines()

# create 3-tupel for every community
# ( number of members | size of overlap | members of community as list )
communities = {k: [len(file[k].replace('\n', '').split(' ')), 0, file[k].replace('\n', '').split(' ')] for k in range(0, len(file))}

#for community in data:
#    community[0] = len(community[2])

In [ ]:
# find overlaps in communities
# looking for the biggest overlap for each community

numberOfOverlaps = 0
for i in range(0, len(communities)):
    for j in range(0, len(communities)):
        if i != j:
            for member in communities[i][2]:
                if member in communities[j][2]:
                    numberOfOverlaps += 1
        if numberOfOverlaps > communities[i][1]:
            communities[i][1] = numberOfOverlaps
        numberOfOverlaps = 0